In [7]:
import gym
import torch
import numpy as np
from torch import nn
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('RLresult/hopper/dqn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
env = gym.make('Hopper-v3') 
print(env.action_space)
print(env.observation_space)

Box([-1. -1. -1.], [1. 1. 1.], (3,), float32)
Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf], (11,), float64)


In [4]:
HIDDEN_SIZE = 32
BATCH_SIZE = 10000
learning_rate = 0.01
HOP_ACTION = 125
CAPACITY = 10000
TRAIN_ITER = CAPACITY*10/BATCH_SIZE
PARAM_PATH = 'DQN_param/hopperDQN.pth'
PARAM_PATH_TEST = 'DQN_param/hopperDQN_test.pth'
#print(Transition(2,1,2,3))
#Transition(state=2, action=1, next_state=2, reward=3)

In [5]:
print(env.observation_space.sample())

[-0.56629454 -3.84583097  0.64303238  0.01604773  1.35994578 -0.00550272
  0.67045608  0.80721827 -1.49249697  0.266348   -0.09017813]


In [8]:
input_state_size = len(env.observation_space.sample())

class DQN(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(DQN, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.ReLU()
        )

    def forward(self, input_element):
        output = self.linear_relu_stack(input_element)
        return output


updatedDQN = DQN(input_state_size, HIDDEN_SIZE, HOP_ACTION).to(device)
baseDQN = DQN(input_state_size, HIDDEN_SIZE, HOP_ACTION).to(device)



In [10]:
updatedDQN.load_state_dict(torch.load(PARAM_PATH))
baseDQN.load_state_dict(updatedDQN.state_dict())

<All keys matched successfully>

In [15]:
class fortest(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(DQN, self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.ReLU()
        )

    def forward(self, input_element):
        output = self.linear_relu_stack(input_element)
        return output
    


SyntaxError: invalid syntax (2056349174.py, line 1)

In [5]:
import random
import numpy as np
np.shape(random.sample(list(range(5)),3))

(3,)

In [7]:
[i for i in range(5)]

[0, 1, 2, 3, 4]

In [11]:
import numpy as np
import random
from collections import namedtuple, deque
from torch.utils.data import Dataset

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))


class EnvData(Dataset):
    
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def __getitem__(self, idx):
        return self.memory[idx]

    def __len__(self):
        return len(self.memory)

    def push(self, *args):
        self.memory.append(Transition(*args))


class CustomDataLoader:

    def __init__(self, dataset, batch_size, suffle):
        self.dataset = dataset
        self.batch_size = batch_size
        self.num_category = len(self.dataset[0])
        self.suffle = suffle
        self.iternum = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.suffle == True:
            index = random.sample(list(range(len(self.dataset))), self.batch_size)
        else:
            index = [i + self.iternum * self.batch_size for i in range(self.batch_size)]
        _batchdata = np.array(self.dataset[index[0]], dtype=np.object)
        _cnum = 0
        while _cnum < self.num_category:
            i = 0
            i = i + 1
            _batchdata[_cnum] = np.array([_batchdata[_cnum]])
            while i < self.batch_size:
                added_data = np.array([self.dataset[index[i]][_cnum]])
                _batchdata[_cnum] = np.concatenate((_batchdata[_cnum], added_data), axis=0)
                i = i + 1
            _cnum = _cnum + 1
        self.iternum = self.iternum + 1
        if (self.iternum + 1) * self.batch_size > len(self.dataset):
            self.iternum = 0
        return _batchdata



In [21]:
    
mycartdata = EnvData(capacity = 3)
#
np_pre_observation = 1
action = 2
observation = 3
reward = 4
done = 5
mycartdata.push(np_pre_observation, action, observation, reward, np.float32(done))
mycartdata.push(np_pre_observation, action, observation, reward, np.float32(done))
mycartdata.push(np_pre_observation, action, observation, reward, np.float32(done))
np_pre_observation = 1
action = 2
observation = 3
reward = 4
done = 6

mycartdata.push(np_pre_observation, action, observation, reward, np.float32(done))
print(mycartdata[1])

Transition(state=1, action=2, next_state=3, reward=4, done=5.0)


In [8]:
print(mycartdata[2])

Transition(state=1, action=2, next_state=3, reward=4, done=6.0)


In [22]:
mydataloader = CustomDataLoader(mycartdata, batch_size=1, suffle = False)

In [13]:
next(iter(mydataloader))

/tmp/ipykernel_2800/3085157848.py:41: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _batchdata = np.array(self.dataset[index[0]], dtype=np.object)


array([array([1, 1]), array([2, 2]), array([3, 3]), array([4, 4]),
       array([5., 5.], dtype=float32)], dtype=object)

In [14]:
next(iter(mydataloader))

/tmp/ipykernel_2800/3085157848.py:41: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _batchdata = np.array(self.dataset[index[0]], dtype=np.object)


array([array([1, 1]), array([2, 2]), array([3, 3]), array([4, 4]),
       array([5., 5.], dtype=float32)], dtype=object)

In [15]:
next(iter(mydataloader))

/tmp/ipykernel_2800/3085157848.py:41: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _batchdata = np.array(self.dataset[index[0]], dtype=np.object)


array([array([1, 1]), array([2, 2]), array([3, 3]), array([4, 4]),
       array([5., 5.], dtype=float32)], dtype=object)

In [23]:
i = 0
while(i < 10):
    print(next(iter(mydataloader)))
    i = i + 1

[array([1]) array([2]) array([3]) array([4]) array([5.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([5.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([6.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([5.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([5.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([6.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([5.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([5.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([6.], dtype=float32)]
[array([1]) array([2]) array([3]) array([4]) array([5.], dtype=float32)]


/tmp/ipykernel_2800/3085157848.py:41: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _batchdata = np.array(self.dataset[index[0]], dtype=np.object)


In [17]:
mycartdata.push(np_pre_observation, action, observation, reward, np.float32(done))
mycartdata.push(np_pre_observation, action, observation, reward, np.float32(done))

In [18]:
print(mycartdata[1])

Transition(state=1, action=2, next_state=3, reward=4, done=6.0)


In [19]:
i = 0
while(i < 10):
    print(next(iter(mydataloader)))
    i = i + 1

[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]
[array([1, 1]) array([2, 2]) array([3, 3]) array([4, 4])
 array([6., 6.], dtype=float32)]


/tmp/ipykernel_2800/3085157848.py:41: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _batchdata = np.array(self.dataset[index[0]], dtype=np.object)


In [24]:
len(mycartdata)

3

In [28]:
total_num = 0
pause = 0
memory_capacity = 5
while total_num < memory_capacity - pause:
    t = 0
    while t < memory_capacity - total_num: #if pg, gain accumulate
        print(t)
        t = t + 1
        if t==3:
            total_num += t
            t = 0
            break
    pause = t
    print("p=",pause)

0
1
2
p= 0
0
1
p= 2
